In [1]:
from openai import OpenAI
import os 
import json 
import sys 
import requests
sys.path.append(r"C:\Users\o\agentflightbooking\fiancialAIAgent")

In [5]:
from openai import OpenAI
deepseek_client = OpenAI(
    api_key="sk-3da1e2817a2942ea8809277679bb05ec",
    base_url="https://api.deepseek.com/v1"
)

In [6]:
def reformuler_question_en_requete_usa(question, client):
    prompt = f"""
You are a legal research assistant for U.S. law.

Your job is to turn a user's natural language question into a focused Google search query
that finds relevant legal documents in PDF format on official U.S. government websites.
and as output give me just the query without explanations

Instructions:
- Use only the key legal terms (state, topic, law name if known).
- End the query with: site:.gov filetype:pdf
- Be concise and relevant.
- give just one query 

Example:
Question: What is the law about privacy rights in California?
Query: california privacy law site:.gov filetype:pdf

---

User question:
\"{question}\"

Google search query:
"""

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content.strip()


In [7]:
question = 'Can a person who travels to cub enter the US?'
reforulation = reformuler_question_en_requete_usa(question=question, client=deepseek_client)

In [8]:
reforulation

'"US entry restrictions Cuba travel site:.gov filetype:pdf"'

In [9]:
import os
import requests

SERPAPI_API_KEY = os.getenv("SERA_API_KEY")

def chercher_pdfs_via_serpapi(query, api_key=SERPAPI_API_KEY):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "num": 5, 
        "filetype": "pdf",
        "hl": "en"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Erreur SerpAPI : {response.status_code} - {response.text}")

    data = response.json()
    results = data.get("organic_results", [])
    pdf_urls = []
    for result in results:
        link = result.get("link", "")
        if link.lower().endswith(".pdf"):
            pdf_urls.append(link)
    return pdf_urls


In [14]:
pdf_link = chercher_pdfs_via_serpapi(query=reforulation)

In [15]:
pdf_link

[]

In [12]:
import os
import requests
from datetime import datetime

def telecharger_pdf_dossier_temp(url_pdf, dossier_temp="data/temp"):
    os.makedirs(dossier_temp, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    nom_fichier = f"document_{timestamp}.pdf"
    chemin_complet = os.path.join(dossier_temp, nom_fichier)
    
    response = requests.get(url_pdf)
    if response.status_code != 200:
        raise Exception(f"Erreur téléchargement PDF : {response.status_code}")
    
    with open(chemin_complet, "wb") as f:
        f.write(response.content)
    
    return chemin_complet


In [13]:
save_pdf = telecharger_pdf_dossier_temp(url_pdf=pdf_link[0])

IndexError: list index out of range

In [ ]:
save_pdf

'data/temp\\document_20250720_122507_153943.pdf'

In [141]:
from pdf2image import convert_from_bytes
from typing import List, Optional
from PIL import Image
import os
import requests
import json
import pytesseract

def ocr(pdf):
    images = convert_from_bytes(pdf)
    ocr_texts = []
    
    for i, image in enumerate(images):
        image_path = f"page_{i + 1}.png"
        image.save(image_path, "PNG")
        print(f"Saved {image_path}")
    images_path = [f"page_{i + 1}.png" for i in range(len(images))]
    for image_path in images_path:
        image = Image.open(image_path)
        with Image.open(image_path) as image:
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                
                text = pytesseract.image_to_string(
                    image, 
                    lang='eng',
                    config='--psm 6'  
                )   
                text = text.replace('\n', ' ').strip()
                ocr_texts.append(text)
        os.remove(image_path)
        print(f"Removed {image_path}")
        
    return ocr_texts

In [142]:
#read pdf 
with open(save_pdf, "rb") as f:
    pdf_bytes = f.read()

In [143]:
ocr_texts = ocr(pdf_bytes)

Saved page_1.png
Saved page_2.png
Removed page_1.png
Removed page_2.png


In [144]:
ocr_texts

[". * oe t ” Tama ne permanent resident B4 a How do I know what —— “Lede: y) \\ > my responsibilities are: f — \\ ns « . > NY i \\ Pc sARTiva cu: hi \\ \\ / Aa US. Citizenship I | d ) and Immigration “| \\ \\ . | / Qs” Services X Ml Everyone in the United States must abide by U.S. law, including Traveling Outside of the United States Federal, State, and local laws and regulations. Permanent residents + A : M 4 : Permanent residence normally permits you to travel outside the who fall fo abide by the laws of the vnited Gpuates may have their United States and to return; however, there are some limitations. ide contains Sone im otant information thot ma help you Lengthy absences, particularly if they involve work or taking up 9 tect and maintai P t resident stat ly nelp y residence abroad, can lead to abandonment and loss of permanent protect and maintain your permanent resident status. residence status, or delayed eligibility for naturalization. Absence for 1 year or more can cause serio

In [145]:
def ai_agent_analyze_ocr(ocr_texts, question,client):

    full_text = "\n\n".join(ocr_texts)
    
    prompt = f"""
You are a legal assistant helping to analyze PDF documents extracted by OCR.

User question: {question}

Below is the extracted text from the PDF document (from OCR):

{full_text}

Please provide a clear and concise answer to the user's question based on the content of the document.
"""
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful legal assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=800
    )
    
    answer = response.choices[0].message.content.strip()
    return answer

In [146]:
final_response = ai_agent_analyze_ocr(ocr_texts,question,client=deepseek_client)

In [147]:
print(final_response)

After obtaining a green card (permanent resident status), you must follow these key procedures to maintain your status and comply with U.S. requirements:  

1. **Abide by U.S. Laws**  
   - Follow all federal, state, and local laws. Criminal convictions may jeopardize your status.  

2. **Tax Obligations**  
   - File federal, state, and local tax returns as required. Filing as a "nonresident alien" or failing to file may be seen as abandoning your status.  

3. **Travel Considerations**  
   - Avoid lengthy absences (especially over 1 year) to prevent abandonment of status.  
   - Obtain a **Reentry Permit** (Form I-131) before traveling abroad for extended periods.  
   - For naturalization eligibility, file **Form N-470** (if eligible) before departing.  

4. **Social Security Number**  
   - Obtain a Social Security card for employment and tax purposes.  

5. **Selective Service Registration**  
   - Males aged 18–26 must register with the Selective Service ([www.sss.gov](https://w

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def semantic_chunking(text, model, threshold=0.5):
    sentences = text.split('.')
    embeddings = model.encode(sentences)
    
    similarities = [np.dot(embeddings[i], embeddings[i+1]) 
                   for i in range(len(embeddings)-1)]
    
    chunks = []
    current_chunk = sentences[0]
    
    for i, sim in enumerate(similarities):
        if sim < threshold:  
            chunks.append(current_chunk)
            current_chunk = sentences[i+1]
        else:
            current_chunk += '. ' + sentences[i+1]
    
    chunks.append(current_chunk)

    return chunks

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\o\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [148]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def semantic_chunking(text, model, threshold=0.5, min_chunk_len=3):

    sentences = [s.strip() for s in text.split('.') if s.strip()]
    if len(sentences) <= 1:
        return sentences  

    embeddings = model.encode(sentences, normalize_embeddings=True)

    similarities = [np.dot(embeddings[i], embeddings[i+1]) for i in range(len(embeddings) - 1)]

    chunks = []
    current_chunk = [sentences[0]]

    for i, sim in enumerate(similarities):
        if sim < threshold and len(current_chunk) >= min_chunk_len:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentences[i + 1]]
        else:
            current_chunk.append(sentences[i + 1])

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks


In [149]:
ocr_text = "\n".join(ocr_texts)
chunks = semantic_chunking(ocr_text, model, threshold=0.5)
chunk_embeddings = model.encode(chunks, normalize_embeddings=True)

In [150]:
chunk_embeddings

array([[-0.01329765,  0.01946899,  0.00136037, ..., -0.01938768,
         0.05344909,  0.0017274 ],
       [-0.00879923,  0.01094613,  0.00468076, ...,  0.04717553,
         0.00682775,  0.01386935],
       [-0.02375646,  0.04887822,  0.00183014, ...,  0.01330889,
         0.03194562,  0.00309651],
       ...,
       [-0.04310621,  0.05244761, -0.00167144, ..., -0.02251903,
        -0.00345487,  0.01512275],
       [-0.01415339, -0.00975431, -0.02446106, ...,  0.01622579,
         0.04869381,  0.05419089],
       [-0.01413745, -0.02300858, -0.00287095, ...,  0.00298377,
         0.01424597,  0.02001162]], dtype=float32)

In [151]:
question_embeddings = model.encode(question,normalize_embeddings=True)

In [152]:
question_embeddings

array([-1.48115596e-02,  4.52744737e-02, -1.24023976e-02, -3.33791077e-02,
        2.07176488e-02, -1.65982905e-03, -8.23159367e-02,  8.16058517e-02,
        3.55976783e-02, -3.59004028e-02, -1.31830852e-02,  5.05585549e-03,
        4.99833375e-02, -2.23481655e-03, -2.68671252e-02,  2.86673699e-02,
       -2.84385728e-03, -5.27732223e-02, -2.20084898e-02, -7.49606714e-02,
       -1.95205621e-02,  3.13361324e-02, -7.21903890e-03, -3.48192453e-02,
       -4.89275716e-02, -1.72795691e-02,  2.28748340e-02,  2.89692562e-02,
        9.41225234e-03,  2.46024108e-03,  2.23541968e-02,  3.15398239e-02,
        3.34690809e-02, -2.33178400e-03,  1.25618624e-06,  2.36877240e-02,
        5.74437976e-02,  2.36924626e-02,  2.11614445e-02, -1.29802979e-03,
        9.53321531e-03, -3.96361016e-02, -2.54641250e-02, -5.34304269e-02,
       -1.00188106e-02, -7.16801956e-02,  1.82917318e-03,  5.18859066e-02,
       -2.63961051e-02, -1.40264779e-02, -1.38266180e-02, -1.22534201e-01,
        2.46729329e-02, -

In [153]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k_chunks(question_embedding, chunks, chunk_embeddings, k=3):
    sims = cosine_similarity([question_embedding], chunk_embeddings)[0]
    top_indices = sims.argsort()[-k:][::-1]
    return [(chunks[i], i, sims[i]) for i in top_indices]


In [154]:
def trouver_page_du_chunk(chunk, ocr_texts):
    for i, page_text in enumerate(ocr_texts):
        if chunk.strip()[:50] in page_text:
            return i + 1  
    return None


In [155]:
top_chunks = retrieve_top_k_chunks(question_embeddings, chunks, chunk_embeddings, k=3)

for chunk, idx, score in top_chunks:
    page = trouver_page_du_chunk(chunk, ocr_texts)
    print(f"Chunk {idx} (score: {score:.2f}) - Page {page if page else 'inconnue'}:\n")
    print(chunk[:500] + "...")


Chunk 12 (score: 0.56) - Page 1:

gov. The Resident Card, please see Customer Guide B2, guide is available in English, Spanish, Chinese, Vietnamese, Tagalog, I Am a Permanent Resident. How Do I....
Chunk 15 (score: 0.50) - Page 2:

Application to Preserve Residence for N-470 Naturalization Purposes You can also visit www. uscis. gov to download forms, e-file some applications, Other U....
Chunk 24 (score: 0.47) - Page inconnue:

B4—I am a permanent resident. How do I know what my responsibilities are? 2 M-607B (October 2013) Y....
